### Importing Libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt


### Choosing the initial Parameters

In [ ]:
# Flight Params


In [2]:
Empty_weights = np.array([1,1.1,1.5,1.6,1.7,1.8,1.7])
Total_weights = np.array([2,3,4,4.5,5,4.5,8])
Empty_weight_ratio = Empty_weights/Total_weights

In [3]:
Empty_weight_ratio

array([0.5       , 0.36666667, 0.375     , 0.35555556, 0.34      ,
       0.4       , 0.2125    ])